<a href="https://colab.research.google.com/github/taka-onohara/google-colab/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Y = rgdpna, K = rkna, L = emp * avh, alpha = 1 - labsh, A = (the rest)
data['Y'] = data['rgdpna']
data['K'] = data['rkna']
data['L'] = data['emp'] * data['avh']
data['alpha'] = 1 - data['labsh']
data['A'] = data['Y'] / ((data['K'] ** data['alpha']) * (data['L'] ** data['labsh']))
#data['result'] = data['A'] * (data['K'] ** data['alpha']) * (data['L'] ** data['labsh'])
data.drop(data.columns[[2,3,4,5,6]], axis=1, inplace=True)

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

print(data.head(10))